In [1]:
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.0
Default GPU Device: /device:GPU:0


In [9]:
import tensorflow as tf
import numpy as np
# import sys

def feedforward_net(D, C):
    # Input layer which is a visible for MNIST images which 28x28x1 = 28*28=784
    X = tf.placeholder(dtype=tf.float32, shape=[None, D]) # num_samples/images x 784, D=784, D=Dimensions of input images
    
    # output classes for MNIST which is based 10 first digits starting from 0 tp 9
    y = tf.placeholder(dtype=tf.float32, shape=[None, C]) # num_samples for output classes x 10, C=10, 10 output classes

    # Connections/synapses from input to the output
    Wxy = tf.Variable(tf.random_normal(shape=[D, C], stddev=1.0, mean=0.0, dtype=tf.float32), dtype=tf.float32)
    
#     # biases which is based on the number of output classes
#     bxy = tf.Variable(tf.zeros(shape=[C]))
    
    # Output probability from the network
    # We can not use relu because we have to generate the probaby for each class
    # Relu only cuts og the negative part but doesn't generate probabilty
    # prob means from 0-1 but the output of relu can be more than one
    # we have to use softmax
    logits = tf.matmul(X, Wxy);
    
    # Neccessary for training for generating the probability values: 0< y<1
    prob = tf.nn.softmax(logits=logits)
    
    # Error between predicted output and ground truth classes/output
    loss = -tf.reduce_mean(y * tf.log(prob))

    return X, y, prob, loss

def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
# tf.contrib.learn.datasets.load_dataset("mnist")
# from tensorflow.contrib.learn.datasets.mnist import input_data

# if __name__ == '__main__':
mnist = input_data.read_data_sets('/home/arasdar/datasets/MNIST_data', one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

D, C = X_train.shape[1], y_train.shape[1]

#     elif net_type == 'ff':
X, y, forward_step, loss = feedforward_net(D, C)

# This is learning rate
alpha = 1e-3
solver = tf.train.RMSPropOptimizer(alpha)
train_step = solver.minimize(loss)

Extracting /home/arasdar/datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/arasdar/datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/arasdar/datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/arasdar/datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
sess = tf.Session()
#     sess.run(tf.initialize_all_variables())

#     WARNING:tensorflow:From 
#     Instructions for updating:
#     Use `tf.global_variables_initializer` instead.
sess.run(tf.global_variables_initializer())

num_epochs = 1000
M = 64 # minibatch
for i in range(num_epochs):
    X_mb, y_mb = mnist.train.next_batch(M)

    _, loss_val = sess.run([train_step, loss], feed_dict={X: X_mb, y: y_mb})

    if i % 100 == 0:
        y_pred = sess.run(forward_step, feed_dict={X: X_val})
        acc = accuracy(y_val, y_pred)

        print('Iter: {} Loss: {:.4f} Validation: {}'.format(i, loss_val, acc))

Iter: 0 Loss: 1.7755 Validation: 0.0534
Iter: 100 Loss: 1.4038 Validation: 0.0524
Iter: 200 Loss: 0.9487 Validation: 0.1124
Iter: 300 Loss: 0.6379 Validation: 0.226
Iter: 400 Loss: 0.4458 Validation: 0.3808
Iter: 500 Loss: 0.2486 Validation: 0.493
Iter: 600 Loss: 0.2811 Validation: 0.5696
Iter: 700 Loss: 0.1902 Validation: 0.6244
Iter: 800 Loss: 0.2066 Validation: 0.6708
Iter: 900 Loss: 0.1487 Validation: 0.7036
